In [1]:
import pandas as pd
import numpy as np
import flask
import pymongo
import json
from datetime import datetime

In [2]:
atlantic_data = pd.read_csv('Atlantic-data/Atlantic_State_Daily.csv')

In [3]:
atlantic_data.columns

Index(['date', 'state', 'positive', 'negative', 'pending',
       'hospitalizedCurrently', 'hospitalizedCumulative', 'inIcuCurrently',
       'inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative',
       'recovered', 'dataQualityGrade', 'lastUpdateEt', 'dateModified',
       'checkTimeEt', 'death', 'hospitalized', 'dateChecked',
       'totalTestsViral', 'positiveTestsViral', 'negativeTestsViral',
       'positiveCasesViral', 'deathConfirmed', 'deathProbable', 'fips',
       'positiveIncrease', 'negativeIncrease', 'total', 'totalTestResults',
       'totalTestResultsIncrease', 'posNeg', 'deathIncrease',
       'hospitalizedIncrease', 'hash', 'commercialScore',
       'negativeRegularScore', 'negativeScore', 'positiveScore', 'score',
       'grade'],
      dtype='object')

columns we want: date, state, positive, negative, hospitalizedCurrently, hospitalizedCumulative, dataQualityGrade, death, deathConfirmed, deathProbable, fips

In [4]:
atlantic_df = atlantic_data[['date', 'state', 'positive', 'negative', 'hospitalizedCurrently', 'hospitalizedCumulative', 'dataQualityGrade', 'death', 'deathConfirmed', 'deathProbable', 'fips']]



In [5]:
atlantic_df

,date,state,positive,negative,hospitalizedCurrently,hospitalizedCumulative,dataQualityGrade,death,deathConfirmed,deathProbable,fips
0,20200731,AK,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2
1,20200731,AL,87723.0,601744.0,1596.0,10521.0,B,1580.0,1531.0,49.0,1
2,20200731,AR,42511.0,460958.0,507.0,2852.0,A,453.0,NaN,NaN,5
3,20200731,AS,0.0,1037.0,NaN,NaN,C,0.0,NaN,NaN,60
4,20200731,AZ,174010.0,759733.0,2302.0,11260.0,A+,3694.0,2431.0,152.0,4
...,...,...,...,...,...,...,...,...,...,...,...
8300,20200126,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8301,20200125,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8302,20200124,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8303,20200123,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53


In [6]:
from datetime import datetime

In [7]:
#atlantic_df['date'] = atlantic_df['date'].strftime('%m/%d/%Y')
#atlantic_df['date'] = datetime.strptime(atlantic_df['date'], '%Y%m%d').strftime('%m/%d/%Y')
atlantic_df['date'] = pd.to_datetime(atlantic_df['date'], format='%Y%m%d')


C:\Users\clemi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
atlantic_df

,date,state,positive,negative,hospitalizedCurrently,hospitalizedCumulative,dataQualityGrade,death,deathConfirmed,deathProbable,fips
0,2020-07-31,AK,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2
1,2020-07-31,AL,87723.0,601744.0,1596.0,10521.0,B,1580.0,1531.0,49.0,1
2,2020-07-31,AR,42511.0,460958.0,507.0,2852.0,A,453.0,NaN,NaN,5
3,2020-07-31,AS,0.0,1037.0,NaN,NaN,C,0.0,NaN,NaN,60
4,2020-07-31,AZ,174010.0,759733.0,2302.0,11260.0,A+,3694.0,2431.0,152.0,4
...,...,...,...,...,...,...,...,...,...,...,...
8300,2020-01-26,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8301,2020-01-25,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8302,2020-01-24,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8303,2020-01-23,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53


In [9]:
state_series = atlantic_df['state']
state_series = state_series.drop_duplicates()

In [10]:
atlantic_list = ['Alaska','Alabama','Arkansas','American Samoa','Arizona','California','Colorado','Connecticut','District of Columbia','Delaware','Florida','Georgia','Guam','Hawaii','Iowa','Idaho','Indiana','Illinois','Kansas','Kentucky','Louisiana','Massachusetts','Maryland','Maine','Michigan','Minnesota','Missouri','Northern Mariana Islands','Mississippi','Montana','North Carolina','North Dakota','Nebraska','New Hampshire','New Jersey','New Mexico','Nevada','New York','Ohio','Oklahoma','Oregon','Pennsylvania','Puerto Rico','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Virginia','Virgin Islands','Vermont','Washington','Wisconsin','West Virginia','Wyoming']
state_series
state_list = state_series.tolist()
state_coords_df=pd.DataFrame(state_list,columns=['State'])


In [11]:
lat=[]
long=[]

In [12]:
import requests
from config import geocode_key

In [13]:
for i in atlantic_list:
    target_url=(f"https://maps.googleapis.com/maps/api/geocode/json?address={i}&key={geocode_key}")
    geodata=requests.get(target_url).json()
    try:
        latitude=geodata['results'][0]['geometry']['location']['lat']
        lat.append(latitude)

        longitude=geodata['results'][0]['geometry']['location']['lng']
        long.append( longitude)
    except:
        lat.apped('NaN')
        long.append('NaN')

In [14]:
state_coords_df['Latitude']=pd.DataFrame(lat)
state_coords_df['Longitude']=pd.DataFrame(long)
state_coords_df['State Name']=pd.DataFrame(atlantic_list)
state_coords_df.head(5)

,State,Latitude,Longitude,State Name
0,AK,64.200841,-149.493673,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AS,-14.270972,-170.132217,American Samoa
4,AZ,34.048928,-111.093731,Arizona


In [15]:
atlantic_df = atlantic_df.rename(columns = {'date':'Date','state':'State','positive':'PositiveTests','negative':'NegativeTests','hospitalizedCurrently':'CurrentlyHospitalized','hospitalizedCumulative':'CumulativeHospitalizations','dataQualityGrade':'DataQualityGrade', 'death':'Deaths','deathConfirmed':'DeathsConfirmed','deathProbable':'DeathsProbable'})
atlantic_df

,Date,State,PositiveTests,NegativeTests,CurrentlyHospitalized,CumulativeHospitalizations,DataQualityGrade,Deaths,DeathsConfirmed,DeathsProbable,fips
0,2020-07-31,AK,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2
1,2020-07-31,AL,87723.0,601744.0,1596.0,10521.0,B,1580.0,1531.0,49.0,1
2,2020-07-31,AR,42511.0,460958.0,507.0,2852.0,A,453.0,NaN,NaN,5
3,2020-07-31,AS,0.0,1037.0,NaN,NaN,C,0.0,NaN,NaN,60
4,2020-07-31,AZ,174010.0,759733.0,2302.0,11260.0,A+,3694.0,2431.0,152.0,4
...,...,...,...,...,...,...,...,...,...,...,...
8300,2020-01-26,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8301,2020-01-25,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8302,2020-01-24,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53
8303,2020-01-23,WA,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,53


In [16]:
atlantic_covid_df=pd.merge(atlantic_df,state_coords_df, on=['State'])

In [17]:
atlantic_covid_df

,Date,State,PositiveTests,NegativeTests,CurrentlyHospitalized,CumulativeHospitalizations,DataQualityGrade,Deaths,DeathsConfirmed,DeathsProbable,fips,Latitude,Longitude,State Name
0,2020-07-31,AK,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
1,2020-07-30,AK,3559.0,221498.0,45.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
2,2020-07-29,AK,3442.0,220222.0,45.0,NaN,A,22.0,22.0,NaN,2,64.200841,-149.493673,Alaska
3,2020-07-28,AK,3350.0,210525.0,44.0,NaN,A,22.0,22.0,NaN,2,64.200841,-149.493673,Alaska
4,2020-07-27,AK,3206.0,204162.0,38.0,NaN,A,21.0,21.0,NaN,2,64.200841,-149.493673,Alaska
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8300,2020-03-11,WY,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming
8301,2020-03-10,WY,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming
8302,2020-03-09,WY,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming
8303,2020-03-08,WY,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming


In [18]:
atlantic_covid_df=atlantic_covid_df.drop('State', axis=1)

In [19]:
atlantic_covid_df = atlantic_covid_df.rename(columns = {'State Name':'State'})
atlantic_covid_df.head()

,Date,PositiveTests,NegativeTests,CurrentlyHospitalized,CumulativeHospitalizations,DataQualityGrade,Deaths,DeathsConfirmed,DeathsProbable,fips,Latitude,Longitude,State
0,2020-07-31,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
1,2020-07-30,3559.0,221498.0,45.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
2,2020-07-29,3442.0,220222.0,45.0,NaN,A,22.0,22.0,NaN,2,64.200841,-149.493673,Alaska
3,2020-07-28,3350.0,210525.0,44.0,NaN,A,22.0,22.0,NaN,2,64.200841,-149.493673,Alaska
4,2020-07-27,3206.0,204162.0,38.0,NaN,A,21.0,21.0,NaN,2,64.200841,-149.493673,Alaska


In [20]:
#atlantic_covid_df['Date']=atlantic_covid_df['Date'].dt.strftime('%m/%d/%Y')

In [21]:
atlantic_covid_df

,Date,PositiveTests,NegativeTests,CurrentlyHospitalized,CumulativeHospitalizations,DataQualityGrade,Deaths,DeathsConfirmed,DeathsProbable,fips,Latitude,Longitude,State
0,2020-07-31,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
1,2020-07-30,3559.0,221498.0,45.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
2,2020-07-29,3442.0,220222.0,45.0,NaN,A,22.0,22.0,NaN,2,64.200841,-149.493673,Alaska
3,2020-07-28,3350.0,210525.0,44.0,NaN,A,22.0,22.0,NaN,2,64.200841,-149.493673,Alaska
4,2020-07-27,3206.0,204162.0,38.0,NaN,A,21.0,21.0,NaN,2,64.200841,-149.493673,Alaska
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8300,2020-03-11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming
8301,2020-03-10,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming
8302,2020-03-09,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming
8303,2020-03-08,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,43.075968,-107.290284,Wyoming


In [22]:
atlantic_covid_latest=atlantic_covid_df.loc[atlantic_covid_df['Date']==atlantic_covid_df['Date'].max(),:]

In [23]:
atlantic_covid_latest

,Date,PositiveTests,NegativeTests,CurrentlyHospitalized,CumulativeHospitalizations,DataQualityGrade,Deaths,DeathsConfirmed,DeathsProbable,fips,Latitude,Longitude,State
0,2020-07-31,3677.0,229429.0,40.0,NaN,A,23.0,23.0,NaN,2,64.200841,-149.493673,Alaska
148,2020-07-31,87723.0,601744.0,1596.0,10521.0,B,1580.0,1531.0,49.0,1,32.318231,-86.902298,Alabama
295,2020-07-31,42511.0,460958.0,507.0,2852.0,A,453.0,NaN,NaN,5,35.201050,-91.831833,Arkansas
443,2020-07-31,0.0,1037.0,NaN,NaN,C,0.0,NaN,NaN,60,-14.270972,-170.132217,American Samoa
581,2020-07-31,174010.0,759733.0,2302.0,11260.0,A+,3694.0,2431.0,152.0,4,34.048928,-111.093731,Arizona
731,2020-07-31,493588.0,7317453.0,7999.0,NaN,B,9005.0,NaN,NaN,6,36.778261,-119.417932,California
881,2020-07-31,46204.0,480142.0,350.0,6413.0,A,1691.0,NaN,NaN,8,39.550051,-105.782067,Colorado
1030,2020-07-31,49810.0,744186.0,69.0,10807.0,B,4432.0,3551.0,881.0,9,41.603221,-73.087749,Connecticut
1177,2020-07-31,12126.0,176615.0,86.0,NaN,A+,585.0,NaN,NaN,11,38.907192,-77.036871,District of Columbia
1326,2020-07-31,14788.0,166259.0,46.0,NaN,A+,585.0,515.0,70.0,10,38.910832,-75.527670,Delaware


In [24]:
json_atlantic_latest_json = atlantic_covid_latest.to_json(orient = 'records')
parsed = json.loads(json_atlantic_latest_json)

In [25]:
with open('atlantic_covid_latest.json', 'w') as f:
    json.dump(parsed, f, indent=4)

In [26]:
from flask_pymongo import PyMongo
from config import mongoURL 

from pymongo import MongoClient
import json
# client = MongoClient('localhost', 27017)
client = MongoClient(mongoURL)

In [27]:
Atlantic_db = client['Atlantic_db']
collection_Atlantic_lt = Atlantic_db['Atlantic_covid_latest']
# # Inserting state data
with open('atlantic_covid_latest.json') as f:
    Atlantic_data1 = json.load(f)
collection_Atlantic_lt.insert_many(Atlantic_data1)

In [28]:
#Daily Data

In [29]:
json_atlantic = atlantic_covid_df.to_json(orient = 'records')
parsed = json.loads(json_atlantic)

In [30]:
with open('atlantic_covid_daily.json', 'w') as f:
    json.dump(parsed, f, indent=4)

In [31]:
Atlantic_db = client['Atlantic_db']
collection_Atlantic = Atlantic_db['Atlantic_covid_daily']
# # Inserting state data
with open('atlantic_covid_daily.json') as f:
    Atlantic_data2 = json.load(f)
collection_Atlantic.insert_many(Atlantic_data2)